In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.functions import monotonically_increasing_id, col, avg

# Initialize SparkSession
spark = SparkSession.builder.appName("THzRandomForest").getOrCreate()

# File paths
filepath_excel = "THz.xlsx"
filepath_csv = "THz.csv"

# Convert Excel to CSV using pandas
import pandas as pd
pd.read_excel(filepath_excel).to_csv(filepath_csv, index=False)

# Load the dataset
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(filepath_csv)

# Handling missing values robustly
for column in df.columns:
    # Check if column is numeric
    if dict(df.dtypes)[column] in ['int', 'double', 'float']:
        # Compute mean, handle case where all values are null
        mean_value = df.select(avg(col(column))).collect()[0][0]
        if mean_value is not None:  # Only fill if mean is computed
            df = df.na.fill({column: mean_value})
        else:
            print(f"Skipping column with all null values: {column}")
    else:
        print(f"Skipping non-numeric column: {column}")

# First 6 columns as input, last 2 columns as output
input_cols = df.columns[:6]
output_cols = df.columns[6:8]

# Combine input features into a single vector
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

# Standardizing the feature set
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withMean=True, withStd=True)

# Splitting the dataset into training and testing sets
(train_data, test_data) = df.randomSplit([0.9, 0.1], seed=42)

# Create a pipeline for preprocessing
pipeline = Pipeline(stages=[assembler, scaler])
pipeline_model = pipeline.fit(train_data)
train_data = pipeline_model.transform(train_data)
test_data = pipeline_model.transform(test_data)

# Add a unique identifier to test data
test_data = test_data.withColumn("id", monotonically_increasing_id())

# Train separate Random Forest models for each output column and store results
results = []
evaluators = {"rmse": "rmse", "mae": "mae", "r2": "r2"}

for target in output_cols:
    print(f"Training model for {target}...")
    
    # Define the regressor
    rf = RandomForestRegressor(featuresCol="scaledFeatures", labelCol=target, numTrees=100, maxDepth=30, seed=42)
    
    # Fit the model
    rf_model = rf.fit(train_data)
    
    # Make predictions and add id column
    predictions = rf_model.transform(test_data).withColumnRenamed("prediction", f"{target} - Predicted")
    
    # Evaluate the model using multiple metrics
    for name, metric in evaluators.items():
        evaluator = RegressionEvaluator(labelCol=target, predictionCol=f"{target} - Predicted", metricName=metric)
        score = evaluator.evaluate(predictions)
        print(f"{metric.upper()} for {target}: {score}")
    
    results.append(predictions.select("id", f"{target} - Predicted", target))

# Combine predictions for both targets into a single DataFrame using 'id'
combined_results = results[0]
for i in range(1, len(results)):
    combined_results = combined_results.join(results[i], on="id", how="inner")

# Rename columns for clarity
combined_results = combined_results.withColumnRenamed(output_cols[0], "PCR - Actual") \
                                   .withColumnRenamed(output_cols[1], "Absorption - Actual")

# Show the final results
print("Random Forest Results:")
combined_results.show()


Skipping non-numeric column: _c9
Training model for PCR偏振效果...
RMSE for PCR偏振效果: 0.14310799018296375
MAE for PCR偏振效果: 0.10876385291858404
R2 for PCR偏振效果: 0.987817581428733
Training model for A吸收效果...
RMSE for A吸收效果: 0.13394015188077835
MAE for A吸收效果: 0.09799217599482053
R2 for A吸收效果: 0.9079270236593961
Random Forest Results:
+---+-----------------------+------------------+---------------------+-------------------+
| id|PCR偏振效果 - Predicted|      PCR - Actual|A吸收效果 - Predicted|Absorption - Actual|
+---+-----------------------+------------------+---------------------+-------------------+
|  0|     3.0318983586493347|3.1159999999999997|   1.0928897558409971| 0.9800000000000004|
|  1|     2.1618164773664614|             2.104|    0.839108733509589| 0.4980000000000002|
|  2|     3.0317769329052635|3.1099999999999994|    2.137128884730524|              2.056|
|  3|     2.1580084968614046|2.0140000000000002|   1.9360633111640175| 1.7890000000000001|
|  4|      3.567774291512706|             3.

In [ ]:
from pyspark.sql.functions import col, avg
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row
import numpy as np
from scipy.optimize import minimize
import warnings

# Suppress warnings globally
warnings.filterwarnings('ignore')

assembler = VectorAssembler(inputCols=input_cols, outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withMean=True, withStd=True)

# Split data into training and test sets
(train_data, test_data) = df.randomSplit([0.9, 0.1], seed=42)

# Pipeline for scaling features
pipeline = Pipeline(stages=[assembler, scaler])
pipeline_model = pipeline.fit(train_data)
train_data = pipeline_model.transform(train_data)
test_data = pipeline_model.transform(test_data)

# Train the Random Forest Regressor
rf = RandomForestRegressor(featuresCol="scaledFeatures", labelCol=output_cols[0], numTrees=100, maxDepth=30)
rf_model = rf.fit(train_data)

# Define the custom input ranges and step sizes
input_ranges = [
    (10, 20, 0.1),   # (min, max, step) for the first feature
    (10, 20, 0.1),   # (min, max, step) for the second feature
    (9, 10, 0.5),    # (min, max, step) for the third feature
    (8, 10, 0.5),    # (min, max, step) for the fourth feature
    (1, 3, 0.5),     # (min, max, step) for the fifth feature
    (16, 18, 0.5)    # (min, max, step) for the sixth feature
]

# Function to generate all combinations of inputs based on defined ranges and step sizes
def generate_input_combinations(input_ranges):
    input_combinations = []
    for (min_val, max_val, step) in input_ranges:
        input_combinations.append(np.arange(min_val, max_val + step, step))
    grid = np.meshgrid(*input_combinations)
    input_combinations = np.vstack([g.ravel() for g in grid]).T
    return input_combinations

# Generate all possible input combinations
input_combinations = generate_input_combinations(input_ranges)

# Initialize variables to track the best input combination and its output
best_input = None
best_output = None
best_sum = -np.inf

# Loop through all input combinations
for inputs in input_combinations:
    # Standardize the inputs before making predictions
    inputs_scaled = pipeline_model.stages[1].transform(
        spark.createDataFrame([Row(features=Vectors.dense(inputs))])
    )

    # Make predictions using the model
    predictions = rf_model.transform(inputs_scaled).select("prediction").collect()[0][0]

    # Calculate the sum of the predictions (PCR + Absorption)
    output_sum = predictions if isinstance(predictions, float) else predictions[0] + predictions[1]

    # Update the best combination if this one is better
    if output_sum > best_sum:
        best_sum = output_sum
        best_input = inputs
        best_output = predictions

# Output results
print("Best input feature combination (denormalized):")
print(best_input)

# Denormalize inputs
scaler_model = pipeline_model.stages[1]
mean_values = scaler_model.mean
std_values = scaler_model.std
best_input_denormalized = [(best_input[i] * std_values[i] + mean_values[i]) for i in range(len(best_input))]

print("Best input feature combination (denormalized):")
print(best_input_denormalized)

print("Prediction for the best input:")
if isinstance(best_output, float):
    print(f"Predicted Output: {best_output}")
else:
    print(f"PCR - Predicted: {best_output[0]}")
    print(f"Absorption - Predicted: {best_output[1]}")